In [ ]:
import pandas as pd
import pyarrow as pa
from pyarrow import parquet as pq
from pyarrow import fs

# 🔐 lakeFS credentials
ACCESS_KEY = "access_key"
SECRET_KEY = "secret_key"
lakefs_endpoint = "http://lakefs-dev:8000/"

# ✅ เชื่อมต่อ FileSystem ของ LakeFS
s3 = fs.S3FileSystem(
    access_key=ACCESS_KEY, 
    secret_key=SECRET_KEY, 
    endpoint_override=lakefs_endpoint
)

# ✅ ระบุ Path หลัก
base_path = 'weather/main/weather.parquet/year=2025/month=5/'

# ✅ วันที่ที่ต้องการดึงข้อมูล
days = [12, 13, 14, 15]

# ✅ สร้าง List เก็บ DataFrame
df_list = []

# ✅ อ่านข้อมูลจากแต่ละวันและรวมเป็น DataFrame
for day in days:
    path = f"{base_path}day={day}/"
    print(f"Loading data from: {path}")
    
    # อ่านข้อมูล Parquet
    dataset = pq.ParquetDataset(path, filesystem=s3)
    
    # อ่านข้อมูล Schema ของ dataset
    schema = dataset.schema
    
    # ✅ แปลง Schema ให้เป็น float64 สำหรับทุกคอลัมน์ที่เป็น float32 หรือ int64
    fields = []
    for i in range(len(schema)):
        field = schema.field(i)
        if isinstance(field.type, pa.FloatArray) or field.type == pa.float32():
            fields.append(pa.field(field.name, pa.float64()))  # แปลงเป็น float64
        elif field.type == pa.int64():  # ตรวจสอบชนิด int64
            fields.append(pa.field(field.name, pa.float64()))  # แปลง int64 เป็น float64
        else:
            fields.append(field)
    
    # สร้าง Schema ใหม่จาก fields ที่แก้ไขแล้ว
    new_schema = pa.schema(fields)
    
    # อ่านข้อมูลและแปลงเป็น DataFrame
    try:
        table = dataset.read().cast(new_schema)
        df = table.to_pandas()
    except Exception as e:
        print(f"Error reading {path}: {e}")
        continue
    
    # ✅ เพิ่ม DataFrame ลงใน List
    df_list.append(df)

# ✅ รวมทุก DataFrame เข้าด้วยกัน
final_df = pd.concat(df_list, ignore_index=True)

# ✅ ตรวจสอบข้อมูล
print(final_df.info())
print(final_df.head())
print(f"Total Records Loaded: {len(final_df)}")